Selección de Criptomonedas:
Mi elección inicial de criptomonedas estará basada en los siguientes criterios:

- Eliminare Las "StableCoins" porque considero que no son de interes para una inversion. 

- Eliminare los outliers de volatilidad alta, ya que presentan mayor riesgo.

- Limitare los datos a las 30 cryptos con mayor capital de mercado. Esto hace que tengan mas "espalda" y las hace mas estables.

- A partir de esto, para mi seleccion final primero voy a elejir las 5 cryptos con mayor market cap.

- En segundo lugar, para mi seleccion final voy a elejir 3 cryptos con el mejor rendimiento el ultimo año.

- Para las ultimas 2 de la seleccion final, voy a seleccionar las 2 cryptos con mayor market cap de las "trending". Es decir las mas buscadas en el ultimo tiempo.


In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests

Obtenemos los datos de la API CoinGecko

In [70]:
response = requests.get('https://api.coingecko.com/api/v3/coins/markets', params={
    'vs_currency': 'usd',
    'order': 'market_cap_desc',
    'limit': 4000,
    'sparkline': False,
    'price_change_percentage': '1h,24h,7d,30d',
    'market_volume': 'usd'
})

data = response.json()
df = pd.DataFrame(data)


Con los datos obtenidos de los cambios porcentuales en las distintas instancias de tiempo, eliminamos las monedas "StableCoin" que no nos interesan para nuestra cartera. 

In [71]:
price_changes = df[['price_change_percentage_1h_in_currency', 'price_change_percentage_24h_in_currency', 'price_change_percentage_7d_in_currency', 'price_change_percentage_30d_in_currency']]
volatilidad = price_changes.std(axis=1)
df['volatility'] = volatilidad
mask = df['volatility'] < 0.3
df = df[~mask]

Eliminamos los outliers de volatilidad alta. Para evitar cryptomonedas de alto riesgo.

In [72]:
Q1 = df['volatility'].quantile(0.25)
Q3 = df['volatility'].quantile(0.75)

IQR = Q3 - Q1
umbral = Q3 + 1.5 * IQR

df = df[df['volatility'] <= umbral]

Limitamos las criptomonedas a las primeras 30 con mayor capital de mercado. 

In [73]:
df = df.sort_values(by='market_cap', ascending=False).head(30)

Obtenemos los porcentajes de cambio en el precio del ultimo año.

In [74]:
import time

price_changes = {}

for index, row in df.iterrows():
    crypto_id = row['id']
    time.sleep(3)  

    historical_data_response = requests.get(f'https://api.coingecko.com/api/v3/coins/{crypto_id}/market_chart', params={
        'vs_currency': 'usd',
        'days': '365',
        'interval': 'daily'
    })

    if historical_data_response.status_code == 200:
        historical_data = historical_data_response.json()
        
        if 'prices' in historical_data and len(historical_data['prices']) > 0:
            initial_price = historical_data['prices'][0][1]
            final_price = historical_data['prices'][-1][1]
            
            price_change_percentage = ((final_price - initial_price) / initial_price) * 100
            price_changes[crypto_id] = price_change_percentage

df['price_change_percentage_1y'] = df['id'].map(price_changes)

In [75]:
missing_data_ids = df[df['price_change_percentage_1y'].isna()]['id'].tolist()

for crypto_id in missing_data_ids:
    time.sleep(3) 

    historical_data_response = requests.get(f'https://api.coingecko.com/api/v3/coins/{crypto_id}/market_chart', params={
        'vs_currency': 'usd',
        'days': '365',
        'interval': 'daily'
    })

    if historical_data_response.status_code == 200:
        historical_data = historical_data_response.json()

        if 'prices' in historical_data and len(historical_data['prices']) > 0:
            initial_price = historical_data['prices'][0][1]
            final_price = historical_data['prices'][-1][1]

            price_change_percentage = ((final_price - initial_price) / initial_price) * 100
            price_changes[crypto_id] = price_change_percentage


df['price_change_percentage_1y'] = df['id'].map(price_changes)
remaining_missing = df['price_change_percentage_1y'].isna().sum()

Obtengo las 5 con mayor market cap y 3 con el mejor rendimiento en el ultimo año.

In [76]:
mayor_market_cap = df.sort_values(by='market_cap', ascending=False).head(5)
mejor_performance = df[~df['id'].isin(mayor_market_cap['id'])].nlargest(3, 'price_change_percentage_1y')
cryptos_elejidas = pd.concat([mayor_market_cap, mejor_performance], ignore_index=True)
cryptos_elejidas = cryptos_elejidas.drop(['volatility','price_change_percentage_1y'],axis=1)

Busco las cryptos en tendencia.

In [77]:
url = "https://api.coingecko.com/api/v3/search/trending"

response = requests.get(url)

if response.status_code == 200:
    trending_coins = response.json().get("coins")
    for index, coin in enumerate(trending_coins):
        coin_name = coin.get("item").get("name")
        coin_symbol = coin.get("item").get("symbol")
        print(f"{index + 1}. {coin_name} ({coin_symbol})")
else:
    print("Error al obtener los datos de tendencia de las monedas.")


1. BeNFT Solutions (BEAI)
2. Hedera (HBAR)
3. HarryPotterObamaSonic10Inu (ETH) (BITCOIN)
4. Unibot (UNIBOT)
5. Rollbit Coin (RLB)
6. Pepe (PEPE)
7. Vulcan Forged (PYR)
8. Worldcoin (WLD)


Obtengo los datos de las cryptos en tendencia.

In [80]:
trending_request = requests.get('https://api.coingecko.com/api/v3/search/trending')
trending_data = trending_request.json().get("coins")

trending_ids = [coin.get("item").get("id") for coin in trending_data]

trending_cryptos = []

for ids in trending_ids:
    crypto_response = requests.get('https://api.coingecko.com/api/v3/coins/markets', params={
        'vs_currency': 'usd',
        'ids': ids,
        'sparkline': False,
        'price_change_percentage': '1h,24h,7d,30d',
        'market_volume': 'usd'
    })
    crypto_data = crypto_response.json()
    trending_cryptos.append(crypto_data[0])

trending_df = pd.DataFrame(trending_cryptos)


[{'id': 'benft-solutions', 'symbol': 'beai', 'name': 'BeNFT Solutions', 'image': 'https://assets.coingecko.com/coins/images/30669/large/BeAI-Coin.png?1686210015', 'current_price': 12.34, 'market_cap': 20986872, 'market_cap_rank': 662, 'fully_diluted_valuation': 86409121, 'total_volume': 250612, 'high_24h': 14.28, 'low_24h': 11.75, 'price_change_24h': -0.9715961068858565, 'price_change_percentage_24h': -7.30056, 'market_cap_change_24h': -1636433.0264952332, 'market_cap_change_percentage_24h': -7.2334, 'circulating_supply': 1700145.796674536, 'total_supply': 7000000.0, 'max_supply': 7000000.0, 'ath': 17.65, 'ath_change_percentage': -30.04773, 'ath_date': '2023-08-08T05:21:55.036Z', 'atl': 1.69, 'atl_change_percentage': 629.4844, 'atl_date': '2023-06-21T23:28:17.791Z', 'roi': None, 'last_updated': '2023-08-15T01:19:29.045Z', 'price_change_percentage_1h_in_currency': -1.8268225288444926, 'price_change_percentage_24h_in_currency': -7.300564644016916, 'price_change_percentage_30d_in_currency

Extraigo las 2 cryptos en tendencia con mayor market_cap y las incluyo en las cryptos elejidas.

In [81]:
trending_df = trending_df.sort_values(by='market_cap', ascending=False)

trending_df = trending_df.head(2)

cryptos_elejidas = pd.concat([cryptos_elejidas, trending_df])
cryptos_elejidas = cryptos_elejidas.reset_index(drop=True)

print(cryptos_elejidas)


                 id symbol               name  \
0           bitcoin    btc            Bitcoin   
1          ethereum    eth           Ethereum   
2       binancecoin    bnb                BNB   
3            ripple    xrp                XRP   
4      staked-ether  steth  Lido Staked Ether   
5               okb    okb                OKB   
6      bitcoin-cash    bch       Bitcoin Cash   
7          litecoin    ltc           Litecoin   
8  hedera-hashgraph   hbar             Hedera   
9              pepe   pepe               Pepe   

                                               image  current_price  \
0  https://assets.coingecko.com/coins/images/1/la...   29423.000000   
1  https://assets.coingecko.com/coins/images/279/...    1845.840000   
2  https://assets.coingecko.com/coins/images/825/...     240.360000   
3  https://assets.coingecko.com/coins/images/44/l...       0.631137   
4  https://assets.coingecko.com/coins/images/1344...    1844.590000   
5  https://assets.coingecko.com/co

In [82]:
trending_df

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated,price_change_percentage_1h_in_currency,price_change_percentage_24h_in_currency,price_change_percentage_30d_in_currency,price_change_percentage_7d_in_currency
1,hedera-hashgraph,hbar,Hedera,https://assets.coingecko.com/coins/images/3688...,0.067152,2213396620,31,3357620810,292947081,0.067256,...,2021-09-15T10:40:28.318Z,9.861110e-03,578.42085,2020-01-02T17:30:24.852Z,None,2023-08-15T01:20:19.740Z,3.411505,10.180957,28.278658,13.736755
5,pepe,pepe,Pepe,https://assets.coingecko.com/coins/images/2985...,0.000001,590271709,71,590271709,126374143,0.000001,...,2023-05-05T16:50:45.302Z,5.514200e-08,2447.09012,2023-04-18T02:14:41.591Z,None,2023-08-15T01:20:23.892Z,-0.413751,3.125793,-15.552810,25.550262


Armo un csv con las cryptos elejidas.

In [83]:
cryptos_elejidas.to_csv('cryptos_elejidas.csv', index=False)